In [16]:
# Importing the speech recognition library
import speech_recognition as sr

In [20]:
# Create a Recognizer instance to process audio input
r=sr.Recognizer()

# Use the default microphone as the audio source
with sr.Microphone() as src:
    print("Listening...")# Notify user that the system is listening
    audio = r.listen(src)# Listen for the first phrase and extract audio data
    audio_input = r.recognize_google(audio)
    print(audio_input)
    audio_input

Listening...
please help me Lag Gai help me


In [1]:
# Function to find pauses based on punctuation in a sentence
def find_pauses(sentence):
    pauses = []# List to store positions of pauses
    pause_chars = ['.', ',', '!', '?', ';', ':'] # Common punctuation indicating pauses
     # Iterate through the sentence character by character
    for i, char in enumerate(sentence):
        if char in pause_chars:# If the character is a pause character
            pauses.append((i, char)) # Add its position and character to the list

    return pauses # Return the list of pauses found


# Example usage of find_pauses function
# Example usage
sentence = "Hello, how are you? I'm fine."
pauses = find_pauses(sentence)
print("Pauses found at positions:", pauses) # Print the detected pauses


Pauses found at positions: [(5, ','), (18, '?'), (28, '.')]


In [2]:
from sklearn.cluster import KMeans# Import KMeans for clustering pauses or delays
import numpy as np# Import KMeans for clustering pauses or delays

# Simulated sentence and corresponding time delays between characters
sentence = "Hello, how are you?"
delays = [0.1, 0.1, 0.2, 0.3, 0.1, 0.7,  # "Hello," # Delays after each character in "Hello,"# Delays in " ho"  # Delays in "w are"
          0.1, 0.1, 0.6,                 # " ho"# Delays in " ho"  # Delays in "w are"
          0.1, 0.2, 0.1, 0.8,            # "w are"  # Delays in "w are"
          0.1, 0.1, 0.1, 0.1, 0.1]       # " you?" # Delays in " you?"

# Convert to 2D array for sklearn
X = np.array(delays).reshape(-1, 1)

# Apply KMeans with 2 clusters: pause vs no-pause
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
labels = kmeans.labels_

# Determine which cluster represents pauses (the one with the higher average delay)
cluster_means = [X[labels == i].mean() for i in range(2)]
pause_cluster = np.argmax(cluster_means)

# Print results
print("Detected pauses at positions (between characters):")
for i, (char, delay, label) in enumerate(zip(sentence, delays, labels)):
    if label == pause_cluster:
        print(f"Pause before '{char}' at position {i} (delay = {delay}s)")


Detected pauses at positions (between characters):
Pause before ',' at position 5 (delay = 0.7s)
Pause before 'o' at position 8 (delay = 0.6s)
Pause before 'r' at position 12 (delay = 0.8s)


In [3]:
import re

# Define a list of common hesitation words
hesitation_words = ['uh', 'uhh', 'uhhh', 'um', 'umm', 'ummm', 'oh', 'ohh', 'ohhh', 'err', 'erm']

# Compile a regex pattern that matches hesitation words (with optional repetition)
pattern = re.compile(r'\b(' + '|'.join(hesitation_words) + r')\b', re.IGNORECASE)

def find_hesitation_words(sentence):
    matches = list(pattern.finditer(sentence))
    results = []
    for match in matches:
        word = match.group()
        position = match.start()
        results.append((word, position))
    return results

# === Example Usage ===
sentence = "I was like uhhh maybe we could, um, try again or ohh just leave it."
hesitations = find_hesitation_words(sentence)

print("Hesitation words found:")
for word, pos in hesitations:
    print(f"'{word}' at position {pos}")


Hesitation words found:
'uhhh' at position 11
'um' at position 32
'ohh' at position 49


In [6]:
import spacy
nlp = spacy.load("en_core_web_md")
# Function to detect if an unexpected word is used in a sentence based on semantic similarity
def detect_semantic_mismatch(sentence, expected_word, actual_word):
    doc = nlp(sentence) # Process the sentence (not used further here, but may be useful for context)
     # Calculate similarity between expected and actual word
    sim = nlp(expected_word).similarity(nlp(actual_word))
    # If similarity is too low, print a warning
    if sim < 0.4:  # threshold based on context
        print(f"Potential word substitution: '{actual_word}' doesn't fit (expected something like '{expected_word}')")


In [31]:
import sounddevice as sd # Library to record audio from the microphone
from scipy.io.wavfile import write # For saving recorded audio as WAV file
import parselmouth # Interface to Praat for speech analysis
from parselmouth.praat import call# Used to make Praat calls
import numpy as np
import tempfile# Used to create temporary files
import os# For file management like deleting temp files

# Function to record audio using the system microphone
def record_audio(duration=5, fs=44100):
    print("Recording... Speak now!")# Prompt the user to begin speaking
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')# Record audio
    sd.wait() # Wait until recording is complete
    print("Recording finished.")  # Notify user that recording is done

    temp_path = tempfile.mktemp(".wav")# Create a temporary WAV file path
    write(temp_path, fs, audio)# Save the recorded audio to the WAV file
    return temp_path # Return the file path for later use

# Function to analyze speech properties using Praat via parselmouth
def analyze_speech(file_path):
    sound = parselmouth.Sound(file_path)# Load the audio file as a sound object

    # Get pitch
    pitch = call(sound, "To Pitch", 0.0, 75, 600)# Extract pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # Compute the mean pitch over the entire sound
    pitch_sd = call(pitch, "Get standard deviation", 0, 0, "Hertz") # Compute standard deviation of pitch
    
    # Get pitch values
    pitch_values = pitch.selected_array['frequency'] # Get the pitch values as a NumPy array
    time_values = pitch.xs()  # Time intervals# Get the corresponding time stamps for the pitch value

    # Count the number of pitch peaks (simple approximation for periods)
    pitch_peaks = np.count_nonzero(pitch_values > 0)  # Count non-zero pitch values (as peaks)

    duration = call(sound, "Get total duration")
    
    # Calculate speech rate based on pitch peaks (approximating syllables/min)
    speech_rate = pitch_peaks / duration * 60  # syllables/min

    # Print results
    print(f"Mean Pitch: {mean_pitch:.2f} Hz")
    print(f"Pitch Variation: ±{pitch_sd:.2f} Hz")
    print(f"Speech Rate: {speech_rate:.2f} syllables/min")

# === Run it ===
audio_path = record_audio(duration=5)  # record 5 seconds
analyze_speech(audio_path)

# Optionally delete the temp file
os.remove(audio_path)


Recording... Speak now!
Recording finished.
Mean Pitch: nan Hz
Pitch Variation: ±nan Hz
Speech Rate: 0.00 syllables/min


In [33]:
# Function to detect hesitation before nouns based on delay threshold
def find_pauses_before_nouns(sentence, delays):
    doc = nlp(sentence) # Process the sentence with spaCy
    for i, token in enumerate(doc): # Iterate over each token in the sentence
        if token.pos_ == "NOUN" and delays[i] > 0.5:# Check if token is a noun and delay exceeds threshold
            print(f"Hesitation before noun '{token.text}' at word index {i} (delay: {delays[i]}s)") # Log pause

# Run hesitation detection on recognized text
print(find_pauses_before_nouns(audio_input, delays))

None


In [34]:
# Function to determine if a sentence is potentially incomplete
def is_sentence_incomplete(sentence):
    doc = nlp(sentence)# Process the sentence with spaCy
    last_token = doc[-1]# Get the last token in the sentence
    if last_token.pos_ in ["CCONJ", "ADP", "VERB"]: # Check for potentially trailing off sentence
        print("Sentence may be incomplete or trailing off.")

# Check if the recognized sentence is incomplete
print(audio_input)


please help me Lag Gai help me


In [2]:
#bag of words finding that a person in problem or not
# Let's create a larger mock dataset to simulate more realistic voice problem detection
import random
import pandas as pd 

# Define mock sentence templates
normal_sentences = [
    "Good morning, how can I help you today?",
    "Everything is fine with my voice.",
    "I spoke at the meeting and felt comfortable.",
    "No issues with speaking or clarity.",
    "My voice feels strong and stable.",
    "I can talk for hours without strain.",
    "Speech is smooth and effortless today.",
    "I'm able to speak clearly without any pain.",
    "I gave a presentation and it went well.",
    "The conversation flowed naturally and easily."
]

problem_sentences = [
    "My throat hurts when I talk.",
    "I feel a lot of strain while speaking.",
    "Sometimes my voice becomes hoarse.",
    "I experience frequent voice breaks.",
    "Talking for too long makes my voice weak.",
    "I often have to clear my throat mid-sentence.",
    "There’s pain when I raise my voice.",
    "My vocal cords feel irritated.",
    "I lose my voice after extended conversations.",
    "My speech sounds raspy and unclear."
]

# Generate mock data
texts = normal_sentences * 5 + problem_sentences * 5  # 50 of each category
labels = ["normal"] * 50 + ["problem"] * 50

# Shuffle the dataset
combined = list(zip(texts, labels))
random.shuffle(combined)
texts, labels = zip(*combined)

# Convert to DataFrame
df_mock = pd.DataFrame({'text': texts, 'label': labels})
df_mock.head()


,text,label
0,I lose my voice after extended conversations.,problem
1,The conversation flowed naturally and easily.,normal
2,Talking for too long makes my voice weak.,problem
3,I experience frequent voice breaks.,problem
4,No issues with speaking or clarity.,normal


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

# Split the mock dataset
X_train, X_test, y_train, y_test = train_test_split(df_mock['text'], df_mock['label'], test_size=0.3, random_state=42)

# Define the Bag of Words pipeline
bow_pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

# Define the TF-IDF pipeline
tfidf_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Train and evaluate BoW model
bow_pipeline.fit(X_train, y_train)
bow_preds = bow_pipeline.predict(X_test)
bow_report = classification_report(y_test, bow_preds, output_dict=True)

# Train and evaluate TF-IDF model
tfidf_pipeline.fit(X_train, y_train)
tfidf_preds = tfidf_pipeline.predict(X_test)
tfidf_report = classification_report(y_test, tfidf_preds, output_dict=True)

# Extract accuracy for comparison
bow_accuracy = bow_report["accuracy"]
tfidf_accuracy = tfidf_report["accuracy"]

{
    "Bag of Words Accuracy": round(bow_accuracy, 2),
    "TF-IDF Accuracy": round(tfidf_accuracy, 2),
    "TF-IDF Recommended": tfidf_accuracy >= bow_accuracy
}


{'Bag of Words Accuracy': 1.0,
 'TF-IDF Accuracy': 1.0,
 'TF-IDF Recommended': True}